In [ ]:
import pandas as pd
import vk_api

In [ ]:
def auth_handler():
    key = input("Enter authentication code: ")
    remember_device = True
    return key, remember_device

# замените своими логином и паролем
login, password = open('../../vk_login_pass.txt', 'r').read().split()

vk_session = vk_api.VkApi(login, password,auth_handler=auth_handler)
try:
    vk_session.auth(token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)

#### Получаем информацию о друзьях
Этот метод соберёт информацию вообще обо всех друзьях

In [ ]:
tools = vk_api.VkTools(vk_session)
# список полей можете поменять самостоятельно, https://dev.vk.com/method/friends.get
friends = tools.get_all_slow(
    'friends.get',
    100,
    {'fields': "nickname,sex,bdate,city,country,has_mobile,contacts,education,universities"}
)

friends = pd.DataFrame(friends['items'])
def f(s): 
    return type(s) == dict and s.get("title")    
    
friends.city = friends.city.apply(f)       
friends.country = friends.country.apply(f)
friends.head()

In [ ]:
friends.to_excel("all_friends.xlsx")

#### Отфильтруем общий список
У меня есть своя база контактов. Я фиксирую людей, с которыми получается общаться лично. Также есть бот, который напоминает поздравлять их с днями рождения. 
В файлике `base.txt` у меня построчно имена знакомых, чьи др мне нужно извлечь.

In [ ]:
with open("base.txt","r", encoding="utf8") as f:
    base = pd.DataFrame(f.read().split("\n"))
base[['last_name','first_name']] = base[0].str.split(expand=True)
base

In [ ]:
contact_base = pd.merge(base,friends, on=['first_name','last_name'], how='left')
contact_base.bdate = pd.to_datetime(
    contact_base.bdate, 
    infer_datetime_format=False, 
    format='%d.%m.%Y', 
    errors='ignore'
)

# Полученные данные я просто вставлю в свою базу
contact_base.to_excel("contact_base.xlsx")

contact_base